#### Use Holoviews to compare HS2 and Ecopuck parameters from stoqs_mb_diamonds

This Notebook is part of the auv-python project (https://github.com/mbari-org/auv-python). It demonstrates how to read and make interactive plots of millions of data points accessed from a STOQS database.

The url in the cell below was generated by going to https://stoqs.shore.mbari.org/stoqs_mb_diamonds/ and clicking the buttons of the Measured Parameters to be included in the Parquet file. Then clicking the "Measured Parameter Data Access" section and clicking the "Estimate requirements" button to verify that the estimated values are within the available values of the server. The cells below load the data into a Pandas DataFrame and makes interactive zoomable plots.

In [1]:
import pandas as pd
import panel.widgets as pnw
import pooch

# Construct the url to retrieve the data from the STOQS database:
parms = [
    "ecopuck_bbp700 (m^-1 sr^-1)",
    "ecopuck_cdom (ppb)",
    "ecopuck_chl (ug/l)",
    "hs2_bbp420 (m-1)",
    "hs2_bbp700 (m-1)",
    "hs2_fl700",
]
    
stoqs_url = "https://stoqs.shore.mbari.org/stoqs_mb_diamonds/api/measuredparameter.parquet?"
stoqs_url += "parameter__name=" + "&parameter__name=".join(parms)
stoqs_url += "&collect=name&include=activity__name"

# Takes several minutes to retrieve the data the first time, thereafter it's read from a local cache
downloader = pooch.HTTPDownloader(timeout=(60, 300))  # (connect, read) timeouts in seconds
file_name = pooch.retrieve(
    url = stoqs_url,
    known_hash="b71a33bc14e52def0119b0269c4dc273e7e223dd49a10c9d018739ecde72b099",
    progressbar=True,
    downloader=downloader,
)
df = pd.read_parquet(file_name)
df.describe()

name,ecopuck_bbp700 (m^-1 sr^-1),ecopuck_cdom (ppb),ecopuck_chl (ug/l),hs2_bbp420 (m-1),hs2_bbp700 (m-1),hs2_fl700
count,3.450397e+06,3.450397e+06,3.450397e+06,678548.000000,678548.000000,6.161929e+06
mean,8.322027e-04,1.497288e+00,2.652438e+00,0.000979,0.004022,7.445026e-04
std,9.314967e-04,1.046130e+00,4.003378e+00,0.002965,0.004004,1.257494e-03
min,9.455070e-05,-1.041218e+00,2.455455e-02,-0.005418,0.000102,-9.492986e-02
25%,3.120663e-04,1.227150e+00,3.117100e-01,-0.000949,0.001260,1.042317e-04
50%,5.455853e-04,1.408950e+00,1.054850e+00,0.000030,0.002628,2.880702e-04
75%,9.768606e-04,1.618020e+00,3.161630e+00,0.001837,0.004921,8.601799e-04
max,6.669172e-03,8.795484e+01,2.982050e+01,0.021402,0.044840,1.075705e-01


In [ ]:
# See that ecopuck data exist at the tail end
df.tail()

In [ ]:
# Use datashader to make interactive biplots of the data
# Do not commit following cell outputs to the repository - they are too big!
import colorcet
import holoviews as hv
from holoviews.operation.datashader import datashade
hv.extension("bokeh")

In [ ]:
# Compare ecopuck and HS2 Chlorophyll
pts_hs2eco = hv.Points(df, ['ecopuck_chl (ug/l)', 'hs2_fl700'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# Compare ecopuck and HS2 420 BB
pts_hs2eco = hv.Points(df, ['ecopuck_bbp700 (m^-1 sr^-1)', 'hs2_bbp420 (m-1)'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# Compare ecopuck BB and HS2 700
pts_hs2eco = hv.Points(df, ['ecopuck_bbp700 (m^-1 sr^-1)', 'hs2_bbp700 (m-1)'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# Ecopuck BB vs. Chl
pts_hs2eco = hv.Points(df, ['ecopuck_bbp700 (m^-1 sr^-1)', 'ecopuck_chl (ug/l)'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# HS2 BB vs. Chl
pts_hs2eco = hv.Points(df, ['hs2_bbp420 (m-1)', 'hs2_fl700'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# Compute proxy_cal_factor for use in 5.2-mpm-bg_biolume-PiO-paper.ipynb and src/data/resample.py
proxy_cal_factor = df['hs2_fl700'].quantile(0.99)
print(f"proxy_cal_factor = {proxy_cal_factor:.6f}")
# proxy_cal_factor = 0.006493